In [ ]:
import torch
import torch.nn as nn
from convolution_lstm import ConvLSTMCell, ConvLSTM
from collections import OrderedDict

In [ ]:
convlstm = ConvLSTM(input_channels=10, hidden_channels=[6, 6, 6], kernel_size=3, batch_first=True)
loss_fn = torch.nn.MSELoss()

input = torch.randn(1, 3, 10, 5, 5)
target = torch.randn(1, 3, 6, 5, 5).double()

output, _ = convlstm(input)
output = output.double()
res = torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)
print(res)

In [ ]:
output.size()

In [ ]:
class VideoAutoencoderLSTM(nn.Module):
    def __init__(self, in_channels=1):
        super(VideoAutoencoderLSTM, self).__init__()
        self.in_channels = in_channels
        self.conv_encoder = nn.Sequential(OrderedDict([
              ('conv1', nn.Conv2d(in_channels=self.in_channels, out_channels=128, kernel_size=11,stride=4, padding=0)),
              ('nonl1', nn.Tanh()),
              ('conv2', nn.Conv2d(in_channels=128, out_channels=64, kernel_size=5,stride=2, padding=0)),
              ('nonl2', nn.Tanh())
            ]))
        self.rnn_enc_dec = ConvLSTM(input_channels=64, hidden_channels=[64, 32, 64], kernel_size=3, batch_first=True)
        self.conv_decoder = nn.Sequential(OrderedDict([
              ('deconv1', nn.ConvTranspose2d(in_channels=64, out_channels=128, kernel_size=5,stride=2, padding=0)),
              ('nonl1', nn.Tanh()),
              ('deconv2', nn.ConvTranspose2d(in_channels=128, out_channels=self.in_channels, kernel_size=11,stride=4, padding=0)),
              ('nonl2', nn.Tanh())
            ]))
    
    def forward(self, x):
        b, t, c, h, w = x.size()
        x = x.view(b*t,c,h,w)
        x = self.conv_encoder(x)
        x = x.view(b, t, x.size(1), x.size(2), x.size(3))
        x, _ = self.rnn_enc_dec(x)
        x = x.view(b*t, x.size(2), x.size(3), x.size(4))
        x = self.conv_decoder(x)
        x = x.view(b, t, x.size(2), x.size(3), x.size(4))
        return x

In [ ]:
model = VideoAutoencoderLSTM()
loss_fn = torch.nn.MSELoss()

input = torch.randn(1, 5, 1, 227, 227)
target = torch.randn(1, 5, 1, 227, 227).double()

output= model(input)
output = output.double()
res = torch.autograd.gradcheck(loss_fn, (output, target), eps=1e-6, raise_exception=True)
print(res)

        model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
    	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
    
    http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm
    https://github.com/blackeagle01/Abnormal_Event_Detection/blob/master/train.py

In [ ]:
from PIL import Image
import torch.utils.data as data
import os
import torchvision.transforms as transforms

class UCSDAnomalyDataset(data.Dataset):
    '''
    Dataset class to load  UCSD Anomaly Detection dataset
    Input: 
    - root_dir -- directory (Train/Test) structured exactly as out-of-the-box folder downloaded from the site
    http://www.svcl.ucsd.edu/projects/anomaly/dataset.htm
    - time_stride (default 1) -- max possible time stride used for data augmentation
    - seq_len -- length of the frame sequence
    '''
    def __init__(self, root_dir, seq_len = 10, time_stride=1, transform=None):
        super(UCSDAnomalyDataset, self).__init__()
        self.root_dir = root_dir
        vids = [d for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d))]
        self.samples = []
        for d in vids:
            for t in range(1, time_stride+1):
                for i in range(1, 200):
                    if i+(seq_len-1)*t > 200:
                        break
                    self.samples.append((os.path.join(self.root_dir, d), range(i, i+(seq_len-1)*t+1, t)))
        self.pil_transform = transforms.Compose([
                    transforms.Resize(227),
                    transforms.Grayscale(),
                    transforms.ToTensor()])
        
    def __getitem__(self, index):
        pref = self.samples[index][0]
        for fr in self.samples[index][1]:
            with open(os.path.join(pref, '{0:03d}.tif'.format(n)), 'rb') as fin:
                frame = Image.open(fin)
                frame = self.pil_transform(frame) / 255.0
                return frame
        

    def __len__(self):
        return len(self.samples)

#img = Image.open(open('./data/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/Train001/001.tif', 'rb'))
#img = img.convert('RGB')
#print(img)
ds = UCSDAnomalyDataset('./data/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train', time_stride=3)
print(ds[0])

In [ ]:
print(len(range(1, 29, 3)))
for i in range(1, 29, 3):
    print(i)

In [ ]:
print(len(range(191, 201)))
for i in range(191,201):
    print(i)

In [1]:
import torchvision

ImportError: /home/mbolonkin/libs/anaconda3/lib/python3.7/site-packages/torchvision/_C.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZN2at7getTypeERKNS_6TensorE